In [ ]:
#papermill parameters
saturn_base="$(homedir())/workspace"
lcfta_base="$(homedir())/work"
base=lcfta_base
dataset_directory = "$(base)/sars-cov2-pangenome-analysis/data/taxon_2697049.genbank"
# dataset_directory = "$(base)/sars-cov2-pangenome-analysis/data/sars-cov-2.annotated.complete"
# dataset_directory = "$(base)/sars-cov2-pangenome-analysis/data/sars-cov-2"

In [ ]:
import Pkg
Pkg.update()

pkgs = [
    "Revise",
    "MetaGraphs",
    "Graphs",
    "JSON"
]

for pkg in pkgs
    try
        eval(Meta.parse("import $pkg"))
    catch
        Pkg.add(pkg)
        eval(Meta.parse("import $pkg"))
    end
end

# Pkg.add(url="https://github.com/cjprybol/Mycelia.git", rev="master")
import Mycelia

In [ ]:
data_directory = joinpath(dataset_directory, "ncbi_dataset", "data")

In [ ]:
fastx_files = filter(x -> occursin("genomic.fna", x), readdir(data_directory, join=true))

In [ ]:
# import BioSequences
# import FASTX
# import Kmers

In [ ]:
graph = Mycelia.initialize_graph()
graph = Mycelia.add_fastx_to_graph!(graph, fastx_files)

In [ ]:
# add fastx metadata to graphs

In [ ]:
data_report_json_file = joinpath(data_directory, "data_report.jsonl")
data_report_json = JSON.parse(open(data_report_json_file))
identifier_to_data_report = Dict(data_report_json["accession"] => data_report_json)
Mycelia.add_metadata_to_graph!(graph, identifier_to_data_report)

In [ ]:
# for (key, value) in graph.vprops[1]
#     println(key)
#     println(value)
#     println()
# end

In [ ]:
function add_biosample_to_graph!(graph, biosample_json)
    try
        graph[biosample_json["identifier"], :identifier]
        @info "node $(biosample_json["identifier"]) already present"
    catch
        Graphs.add_vertex!(graph)
        vertex_id = Graphs.nv(graph)

        MetaGraphs.set_prop!(graph, vertex_id, :TYPE, "BIOSAMPLE")
        for (key, value) in biosample_json
            MetaGraphs.set_prop!(graph, vertex_id, Symbol(key), value)
        end
    end
    return graph
end

function add_biosamples_to_graph!(graph, biosample_json_file)
    open(biosample_json_file) do io
        # for i in 1:1
            # line = readline(io)
        for line in eachline(io)
            jsonl = JSON.parse(line)
            jsonl["attributes"] = Dict(attribute["name"] => attribute["value"] for attribute in jsonl["attributes"])
            jsonl["identifier"] = jsonl["accession"]
            add_biosample_to_graph!(graph, jsonl)
        end
    end
end

In [ ]:
biosample_json_file = joinpath(data_directory, "biosample.jsonl")
add_biosamples_to_graph!(graph, biosample_json_file)
# @time biosample_json = JSON.parse(open(biosample_json_file))

In [ ]:
# kmer_size = Mycelia.assess_dnamer_saturation(fastx_files)